In [8]:
import pandas as pd
import numpy as np
import json
import os
pd.set_option('display.max_columns', 256)
pd.set_option('display.max_rows', 512)
from matplotlib import pyplot as plt

from collections import Counter
pd.set_option('display.max_rows', 512)
pd.set_option('display.max_columns', 512)


import shapely
import geopandas as gpd
import h3
import requests
import math
from geopy import distance

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

from catboost import CatBoostRegressor, Pool


### Вернемся к датасету недвижимости
### Задача - научиться предсказывать цену 
### Метрика - MAPE

In [ ]:
realty_data = pd.read_csv('data/realty_data.csv')
realty_data.head(5)

### отложим треть выборки

In [ ]:
realty_data['split_city'] = realty_data.apply(lambda t: str(t.city)+'/'+str(t.settlement), axis=1)

realty_train, realty_val, _, _ = train_test_split(
    realty_data.drop(['split_city'], axis=1),
    realty_data[[]],
    test_size=0.33,
    random_state=42,
    stratify=realty_data.split_city
)

y_val = realty_val.price

print(realty_train.shape)
print(realty_val.shape)


### Бейзлайн 1

### Будем предсказывать итоговую цену как произведение средней цены локации (города + района) и площади

In [ ]:
avg_price_location = realty_train.fillna('N/A').groupby(
    ['city', 'settlement', 'district'], dropna=False
)[['price', 'total_square']].agg(sum).reset_index()

avg_price_location['avg_price'] = avg_price_location['price']/avg_price_location['total_square']

naive_price_location = dict(
    avg_price_location.apply(lambda t: ((t.city, t.settlement, t.district), t.avg_price), axis=1).values
)


In [ ]:
y_pred1 = realty_val[['city', 'settlement', 'district', 'total_square']].fillna('N/A').apply(
    lambda t: naive_price_location.get((t.city, t.settlement, t.district))*t.total_square, axis=1
)


In [ ]:
print('MAE is: {value:,}'.format(value=round(mean_absolute_error(y_val, y_pred1))))
print('MAPE is : {value:.2f}%'.format(value=100.0*mean_absolute_percentage_error(y_val, y_pred1)))

### Бейзлайн 2

### Попробуем сделать то же самое, но агрегируя до почтового индекса


In [ ]:
postcode_df = realty_train[['postcode', 'total_square']]

# будем определять цену только если количество предложений не меньше 5
postcode_count = dict(postcode_df.groupby('postcode').size().reset_index().values)
postcodes = set(key for key, value in postcode_count.items() if value >= 5)


In [ ]:
postcode_df['group_index'] = postcode_df.postcode.apply(lambda t: t if t in postcodes else -1)
postcode_df['price'] = realty_train.price


In [ ]:
avg_price_postcode = postcode_df.groupby('group_index')[['price', 'total_square']].agg(sum).reset_index()
avg_price_postcode['avg_price'] = avg_price_postcode['price']/avg_price_postcode['total_square']

naive_price_postcode = dict(zip(avg_price_postcode.group_index, avg_price_postcode.avg_price))
default_naive_price = naive_price_postcode.get(-1)


In [ ]:
y_pred2 = realty_val.apply(
    lambda t: naive_price_postcode.get(t.postcode, default_naive_price)*t.total_square, axis=1
)


In [ ]:
print('MAE is: {value:,}'.format(value=round(mean_absolute_error(y_val, y_pred2))))
print('MAPE is : {value:.2f}%'.format(value=100.0*mean_absolute_percentage_error(y_val, y_pred2)))


### бейзлайн 3

### Бустинг на (тех же) 5 фичах + количество комнат и этаж

In [ ]:
train_num_cols = ['total_square'] + ['rooms', 'floor']
train_cat_cols = {
    'city': 'N/A',
    'settlement': 'N/A',
    'district': 'N/A',
    'postcode': 0.0
}

In [ ]:
catboost_df = realty_train[list(train_cat_cols.keys()) + train_num_cols].copy()
catboost_eval_df = realty_val[list(train_cat_cols.keys()) + train_num_cols].copy()

#заполним пропущенные значения
for feature, value in train_cat_cols.items():
    catboost_df[feature] = catboost_df[feature].fillna(value)
    catboost_eval_df[feature] = catboost_eval_df[feature].fillna(value)
    if catboost_df[feature].dtypes != 'O':
        catboost_df[feature] = catboost_df[feature].astype('int')
        catboost_eval_df[feature] = catboost_eval_df[feature].astype('int')


In [ ]:
train_index, test_index, _, _ = train_test_split(
    catboost_df.index,
    catboost_df[[]],
    test_size=0.2
)


In [ ]:
pool_train = Pool(
    catboost_df.loc[train_index],
    realty_train.price.loc[train_index],
    cat_features = list(train_cat_cols.keys())
)
pool_test = Pool(
    catboost_df.loc[test_index],
    realty_train.price.loc[test_index],
    cat_features = list(train_cat_cols.keys())
)

pool_val = Pool(
    catboost_eval_df,
    realty_val,
    cat_features = list(train_cat_cols.keys())
)


In [ ]:
model = CatBoostRegressor(
    depth=4,
    iterations=5000,
    learning_rate=0.2,
    loss_function='MAE',
    verbose=250,
    od_type='IncToDec',
    od_pval=0.1
)
model.fit(pool_train, eval_set=pool_test)

In [ ]:
y_pred3 = model.predict(pool_val)

In [ ]:
print('MAE is: {value:,}'.format(value=round(mean_absolute_error(y_val, y_pred3))))
print('MAPE is : {value:.2f}%'.format(value=100.0*mean_absolute_percentage_error(y_val, y_pred3)))


### Теперь попробуем собрать фичи

### пример - супермаркеты

### достанем минимальную/максимальную широту/долготу - нужно ограничить запрос

In [ ]:
def get_utm_code(lat: float, lon:float):
    utm_band = int((lon+180) / 6) % 60 + 1
    utm_band = f'0{utm_band}'[-2:]
    utm  = ('EPSG:326' if lat > 0 else 'EPSG:327') + utm_band
    return utm


In [ ]:
min_lat = realty_data.lat.min()
max_lat = realty_data.lat.max()
min_lon = realty_data.lon.min()
max_lon = realty_data.lon.max()

print(min_lat, max_lat, min_lon, max_lon)

### сдвинем на 1000 метров наружу

In [ ]:
# сдвинем на 1000 метров
bbox = [
    (min_lon, min_lat),
    (min_lon, max_lat),
    (max_lon, max_lat),
    (max_lon, min_lat),
    (min_lon, min_lat)
]
bbox = shapely.geometry.Polygon([shapely.geometry.Point(lon, lat) for lon, lat in bbox])

bbox = gpd.GeoDataFrame({'type':['bbox'], 'geometry': [bbox]})
bbox = bbox.set_crs('EPSG:4326')

centroid_lon, centroid_lat = list(bbox.geometry.iloc[0].centroid.coords)[0]
bbox = bbox.to_crs(get_utm_code(centroid_lon, centroid_lat))

bbox['geometry'] = bbox['geometry'].apply(lambda t: t.buffer(1000.0))

bbox = bbox.to_crs('EPSG:4326')
bbox['geometry'] = bbox['geometry'].apply(lambda t: t.envelope)


In [ ]:
min_lat = min([lat for lon, lat in bbox.geometry.iloc[0].convex_hull.exterior.coords])
max_lat = max([lat for lon, lat in bbox.geometry.iloc[0].convex_hull.exterior.coords])
min_lon = min([lon for lon, lat in bbox.geometry.iloc[0].convex_hull.exterior.coords])
max_lon = max([lon for lon, lat in bbox.geometry.iloc[0].convex_hull.exterior.coords])

print(min_lat, max_lat, min_lon, max_lon)

In [ ]:
bbox_constraint = f'({min_lat},{min_lon},{max_lat},{max_lon})'

### запросим все магазины [по тэгу shop](https://wiki.openstreetmap.org/wiki/Key:shop)

In [ ]:
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

query = """
[out:json][timeout:300];
(
node[shop~".*"]{bbox_constraint};
);
out;
""".format(bbox_constraint = bbox_constraint)

response = requests.get(url, params={'data': query})
data = response.json()

print(len(data['elements']))


In [ ]:
shops = pd.DataFrame(
    [
        (
            int(elem.get('id')),
            elem.get('tags').get('shop'),
            elem.get('tags').get('name'),
            elem.get('tags').get('brand'),
            float(elem.get('lat')),
            float(elem.get('lon')),
        )
        for elem in data['elements']
    ],
    columns = ['id', 'shop', 'name', 'brand', 'lat', 'lon']
)

shops.head()

### по типу магазинов

In [ ]:
shops.groupby('shop', dropna=False).size().reset_index().\
rename({0:'count'}, axis=1).sort_values('count', ascending=False)

### запросим все poi [по тэгу amenity](https://wiki.openstreetmap.org/wiki/Key:amenity)

In [ ]:
url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

query2 = """
[out:json][timeout:300];
(
node[amenity~".*"]{bbox_constraint};
);
out;
""".format(bbox_constraint = bbox_constraint)

response2 = requests.get(url, params={'data': query2})
data2 = response2.json()

print(len(data2['elements']))


In [ ]:
amenities = pd.DataFrame(
    [
        (
            int(elem.get('id')),
            elem.get('tags').get('amenity'),
            elem.get('tags').get('name'),
            elem.get('tags').get('brand'),
            float(elem.get('lat')),
            float(elem.get('lon')),
        )
        for elem in data2['elements']
    ],
    columns = ['id', 'amenity', 'name', 'brand', 'lat', 'lon']
)

amenities.head()

### по типу poi

In [ ]:
amenities.groupby('amenity', dropna=False).size().reset_index().\
rename({0:'count'}, axis=1).sort_values('count', ascending=False)

### добавим фичей - посчитаем количество объектов вокруг

In [ ]:
RES = 8

shop_categories = [
    'convenience', 'supermarket', 'beauty', 'clothes', 'hairdresser', 'alcohol', 'outpost'
]

amenity_categories = [
    'pharmacy', 'cafe', 'fast_food', 'atm', 'restaurant', 'bank'
]


### соберем все в словарь по индексу для быстрого поиска

In [ ]:
index_dict = {}

for category in shop_categories:
    for lat, lon in shops[shops.shop==category][['lat', 'lon']].values:
        h3_index = h3.geo_to_h3(lat, lon, RES)
        objects = index_dict.get(h3_index, {})
        objects[category] = objects.get(category, []) + [(lat, lon)]
        index_dict[h3_index] = objects
        
for category in amenity_categories:
    for lat, lon in amenities[amenities.amenity==category][['lat', 'lon']].values:
        h3_index = h3.geo_to_h3(lat, lon, RES)
        objects = index_dict.get(h3_index, {})
        objects[category] = objects.get(category, []) + [(lat, lon)]
        index_dict[h3_index] = objects

### достаем объекты по индексу точки и соседним индексам, фильтруем по расстоянию

In [ ]:
def get_num_objects(
    lat:float,
    lon:float,
    category:str,
    max_dist:float,
    index_dict:dict = index_dict,
    RES:int = RES
):
    edge_length = h3.edge_length(RES)*1000
    num_rings = math.ceil(max_dist/(2*edge_length))

    all_objects_cnt = sum([
        1
        for hex_id in h3.k_ring(h3.geo_to_h3(lat, lon, RES), num_rings)
        for obj_lat, obj_lon in index_dict.get(hex_id, {}).get(category, [])
        if distance.great_circle((lat, lon), (obj_lat, obj_lon)).m <= max_dist
    ])

    return all_objects_cnt



### добавим колонки в оба датафрейма

In [ ]:
for category in shop_categories + amenity_categories:

    get_num_objects_lambda = lambda row: get_num_objects(row[0], row[1], category, 500.0)

    realty_train[category] = np.apply_along_axis(
        get_num_objects_lambda, 1, realty_train[['lat', 'lon']].values
    )
    realty_val[category] = np.apply_along_axis(
        get_num_objects_lambda, 1, realty_val[['lat', 'lon']].values
    )



### зафитим модель 

In [ ]:
train_num_cols = ['total_square'] + shop_categories + amenity_categories:
train_cat_cols = {
    'city': 'N/A',
    'settlement': 'N/A',
    'district': 'N/A',
    'postcode': 0.0
}

In [ ]:
catboost_df = realty_train[list(train_cat_cols.keys()) + train_num_cols].copy()
catboost_eval_df = realty_val[list(train_cat_cols.keys()) + train_num_cols].copy()

#заполним пропущенные значения
for feature, value in train_cat_cols.items():
    catboost_df[feature] = catboost_df[feature].fillna(value)
    catboost_eval_df[feature] = catboost_eval_df[feature].fillna(value)
    if catboost_df[feature].dtypes != 'O':
        catboost_df[feature] = catboost_df[feature].astype('int')
        catboost_eval_df[feature] = catboost_eval_df[feature].astype('int')


In [ ]:
train_index, test_index, _, _ = train_test_split(
    catboost_df.index,
    catboost_df[[]],
    test_size=0.2
)


In [ ]:
pool_train = Pool(
    catboost_df.loc[train_index],
    realty_train.price.loc[train_index],
    cat_features = list(train_cat_cols.keys())
)
pool_test = Pool(
    catboost_df.loc[test_index],
    realty_train.price.loc[test_index],
    cat_features = list(train_cat_cols.keys())
)

pool_val = Pool(
    catboost_eval_df,
    realty_val,
    cat_features = list(train_cat_cols.keys())
)


In [ ]:
model = CatBoostRegressor(
    depth=4,
    iterations=5000,
    learning_rate=0.2,
    loss_function='MAE',
    verbose=250,
    od_type='IncToDec',
    od_pval=0.1
)
model.fit(pool_train, eval_set=pool_test)

In [ ]:
y_pred3 = model.predict(pool_val)

In [ ]:
print('MAE is: {value:,}'.format(value=round(mean_absolute_error(y_val, y_pred3))))
print('MAPE is : {value:.2f}%'.format(value=100.0*mean_absolute_percentage_error(y_val, y_pred3)))


In [ ]:
train_num_cols2 = ['total_square'] + ['rooms', 'floor'] + shop_categories + amenity_categories
train_cat_cols2 = {
    'city': 'N/A',
    'settlement': 'N/A',
    'district': 'N/A',
    'postcode': 0.0
}


In [ ]:
catboost_df2 = realty_train[list(train_cat_cols2.keys()) + train_num_cols2].copy()
catboost_eval_df2 = realty_val[list(train_cat_cols2.keys()) + train_num_cols2].copy()

#заполним пропущенные значения
for feature, value in train_cat_cols2.items():
    catboost_df2[feature] = catboost_df2[feature].fillna(value)
    catboost_eval_df2[feature] = catboost_eval_df2[feature].fillna(value)
    if catboost_df2[feature].dtypes != 'O':
        catboost_df2[feature] = catboost_df2[feature].astype('int')
        catboost_eval_df2[feature] = catboost_eval_df2[feature].astype('int')


In [ ]:
train_index2, test_index2, _, _ = train_test_split(
    catboost_df2.index,
    catboost_df2[[]],
    test_size=0.2
)


In [ ]:
pool_train2 = Pool(
    catboost_df2.loc[train_index],
    realty_train.price.loc[train_index],
    cat_features = list(train_cat_cols.keys())
)
pool_test2 = Pool(
    catboost_df2.loc[test_index],
    realty_train.price.loc[test_index],
    cat_features = list(train_cat_cols.keys())
)

pool_val2 = Pool(
    catboost_eval_df2,
    realty_val,
    cat_features = list(train_cat_cols.keys())
)


In [ ]:
model2 = CatBoostRegressor(
    depth=4,
    iterations=5000,
    learning_rate=0.2,
    loss_function='MAE',
    verbose=250,
    od_type='IncToDec',
    od_pval=0.1
)
model2.fit(pool_train2, eval_set=pool_test2)

In [ ]:
y_pred4 = model2.predict(pool_val2)


In [ ]:
print('MAE is: {value:,}'.format(value=round(mean_absolute_error(y_val, y_pred4))))
print('MAPE is : {value:.2f}%'.format(value=100.0*mean_absolute_percentage_error(y_val, y_pred4)))


In [ ]:
plt.figure(figsize=(16,9))
plt.barh(
    model2.feature_names_,
    model2.feature_importances_
)
plt.title('feature importances')
plt.xlabel('feature importance')
plt.ylabel('feature name')
plt.show()
